## MSCI 332 Assignment 4 - Inventory Control with Shortage

### Initializing constants and randomized values

In [1]:
import random
import time
from statistics import stdev

# setting the seed to the last four digits of matt's student number
random.seed(6975)

# T can be 10, 20 or 30
T                  = 4
num_replications   = 1
storageCapacity    = 40
orderCapacity      = 40

# print(f"ordering cost: {orderingCost}, holdingRate: {holdingRate}, shortageRate: {shortageRate}")

In [2]:
# initializing our two matrices - item production and obj. fn value
item_production = [[0 for i in range(storageCapacity+1)] for j in range(T+1)]
f = [[0 for i in range(storageCapacity+1)] for j in range(T+2)]
timing_data = []
inventory_levels = [0] * T
total_costs = [0] * num_replications
holding_costs = []
ordering_costs = []
shortage_costs = []

for k in range(num_replications):
    # initializing the random integer demand values
    
    start_time = time.time()
    total_shortages = 0
    total_shortage_cost = 0
    total_order_costs = 0
    total_orders = 0
    
    demands = [0] * (T + 1)
    holding_rates = [0] * (T + 1)
    ordering_rates = [0] * (T + 1)
    shortage_rates = [0] * (T + 1)
    counter = 1
    for stage in range(T , 0, -1):
        demand = random.randint(0 , 10)
        demands[counter] = demand
        orderingCost = round(random.uniform(2 , 20), 1)
        holdingRate  = round(random.uniform(5 , 15), 1)
#         orderingCost = 1
#         holdingRate = 0.5
        holding_rates[counter] = holdingRate
        ordering_rates[counter] = orderingCost
        
        # change these bounds depending on the shortage rate that you're testing
        # high - [15,35], medium - [10,25], low - [5,15]
        shortageRate = round(random.uniform(15 , 35), 1)
#         shortageRate = 11.1
        print(f"shortage rate: {shortageRate}")
        shortage_rates[counter] = shortageRate
        counter += 1
        print(shortage_rates)
        print(ordering_rates)
        print(holding_rates)

        for i in range(storageCapacity + 1):
            minOrder = 0
            maxOrder = min(orderCapacity, storageCapacity - i + demand)

            value = float("inf")

            for orderedItems in range(minOrder, maxOrder + 1):
                productionCost = holdingCost = shortageCost = 0    

                surplus = i + orderedItems - demand
#                 print(surplus)
#                 print(orderingCost)
#                 print(shortageRate)

                # if there's a shortage, it means we carry forward 0 units.
                if surplus < 0 :
                    surplus = 0

                if orderedItems < demand - i:
                    shortageCost += shortageRate * (demand - i - orderedItems)
                    holdingCost = 0

                elif orderedItems > demand - i:
                    holdingCost += holdingRate * (i + orderedItems - demand)
                    shortageCost = 0

                orderCost = orderingCost * orderedItems
                if orderCost > 0:
                    total_order_costs += orderCost
                    total_orders += 1

                challengerValue = holdingCost + shortageCost + orderCost + f[stage+1][surplus]

                if challengerValue < value :
                    value = challengerValue
                    bestMove = orderedItems

            f[stage][i] = value
            item_production[stage][i] = bestMove   

    print(f"for run: {k}")
    print("Optimal cost is " + str(f[1][0]))
    total_costs[k] = f[1][0]

    solutionString = "Production amounts:"
    i = shortages = 0
    produced = []
    total_holding_costs_rep = []
    total_ordering_costs_rep = []
    total_shortage_costs_rep = []
    print(total_shortage_costs_rep)
    print(f"f ={f}")
    for stage in range(1,T+1) :
        if i < 0:
            i = 0
        inventory_levels[stage - 1] += i 
        solutionString += " " + str(item_production[stage][i])
        produced.append(item_production[stage][i])
        shortage = demands[stage] - item_production[stage][i] - i
        print(f"demand={demands[stage]}, produced={item_production[stage][i]}, shortage={max(shortage,0)}, inv={i}")
        
        total_shortage_costs_rep.append(shortage * shortage_rates[stage]) if shortage > 0 else total_shortage_costs_rep.append(0)
        
        i = i + item_production[stage][i] - demands[stage]
        if i < 0:
            i = 0
        total_holding_costs_rep.append(i * holding_rates[stage])
        total_ordering_costs_rep.append(item_production[stage][i] * ordering_rates[stage])

    
    print(solutionString)
    print(produced)
    running_time = time.time() - start_time;
    timing_data.append(running_time)
    holding_costs.append(total_holding_costs_rep)
    ordering_costs.append(total_ordering_costs_rep)
    shortage_costs.append(total_shortage_costs_rep)
    
    print("\n\n--- %s seconds ---\n\n" % (running_time))

cum_timing = 0.0
for i in range (num_replications):
    cum_timing += timing_data[i]

average_inventory_levels = [total_level / T for total_level in inventory_levels]
#TODO: holding costs, inventory costs, shortageCosts

total_holding_cost = 0
total_ordering_cost = 0
total_shortage_cost = 0

print("total_costs")
print(total_costs)

for i in holding_costs:
    for j in i:
        if j > 0:
            total_holding_cost += j

for i in ordering_costs:
    for j in i:
        if j > 0:
            total_ordering_cost += j

for i in shortage_costs:
    for j in i:
        if j > 0:
            total_shortage_cost += j
            
final = total_shortage_cost + total_ordering_cost + total_holding_cost
print(f"final: {final}")

print(holdingRate)
print(f"HOLDING COSTS = {holding_costs}")
print(f"SHORTAGE COSTS = {shortage_costs}")
print(f"ORDERING COSTS = {ordering_costs}")
print(f"For T = {T}")
print(final/10)
print(f"average cost: {round((final / num_replications), 2)}")
# print(f"cost standard deviation: {round(stdev(total_costs), 2)}")
print(f"average inventory levels at beginning of every stage: {average_inventory_levels}")
print(f"average holding costs per run: {round(total_holding_cost / num_replications, 2)}")
print(f"average ordering costs per run: {round(total_ordering_cost / num_replications, 2)}")
print(f"average shortage costs per run: {round(total_shortage_cost / num_replications, 2)}")
print(f"average dp program running time: {round(cum_timing / num_replications, 8)} seconds")
print('----------------')
print(total_costs)
print(total_holding_cost)
print(total_ordering_cost)
print(total_shortage_cost)

shortage rate: 20.7
[0, 20.7, 0, 0, 0]
[0, 13.8, 0, 0, 0]
[0, 14.2, 0, 0, 0]


TypeError: list indices must be integers or slices, not float

In [ ]:
%debug